<a href="https://colab.research.google.com/github/eduardodut/Analise-tweets-ICD-Unifor/blob/master/AnaliseTweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Leitura dos dados

In [5]:
def downloadJsonFiles():
  # Fetch a single <1MB file using the raw GitHub URL.
  !curl --remote-name \
      -H 'Accept: application/vnd.github.v3.raw' \
      --location https://raw.githubusercontent.com/eduardodut/Analise-tweets-ICD-Unifor/master/Dados/LulaOficial.json


  !curl --remote-name \
      -H 'Accept: application/vnd.github.v3.raw' \
      --location https://raw.githubusercontent.com/eduardodut/Analise-tweets-ICD-Unifor/master/Dados/jairbolsonaro.json     

downloadJsonFiles()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 47.6M  100 47.6M    0     0  54.9M      0 --:--:-- --:--:-- --:--:-- 54.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25.9M  100 25.9M    0     0  52.1M      0 --:--:-- --:--:-- --:--:-- 52.1M


In [0]:
import pandas as pd

def carregarDados():
  import json
  import pandas as pd
  dataset_lula = pd.DataFrame()
  #tweetsLula = pd.read_json('/content/LulaOficial.json', lines = True)
  with open('/content/LulaOficial.json') as lula_file:
    dados = json.load(lula_file)
    dataset_lula = pd.DataFrame(dados)


  dataset_bolso = pd.DataFrame()
  #tweetsLula = pd.read_json('/content/LulaOficial.json', lines = True)
  with open('/content/jairbolsonaro.json') as bolso_file:
    dados = json.load(bolso_file)
    dataset_bolso = pd.DataFrame(dados)

  return dataset_bolso, dataset_lula

dataset_bolso, dataset_lula = carregarDados()


In [0]:
def cleaning_data(dataset):

  #realiza cópia do dataset
  dataset_temp = dataset.copy()
  
  #conversão da coluna 'created_at' para formato de data
  dataset_temp['created_at'] = dataset_temp['created_at'].astype('datetime64[ns]')

  #criação da coluna 'caracter_count' a partir da coluna 'display_text_range', que indica o tamanho do tweet
  dataset_temp['character_count'] = [(dataset_temp['display_text_range'][i][1]-dataset_temp['display_text_range'][i][0] + 1)for i in range(0,dataset_temp.shape[0])]

  #função que extrai dados do dicionário da coluna 'user' e transforma em uma coluna do dataset
  def extracao_dados_user(coluna):
    if coluna not in dataset_temp.columns:
      dataset_temp[coluna] = [pd.DataFrame(dataset_temp['user']).loc[i][0].get(coluna) for i in range(0,dataset_temp.shape[0])]

  #Extração de dados de seguidores a cada tweet emitido
  #removido pois não parece seguir  linha temporal dos tweets
  #extracao_dados_user('followers_count')

  #Número de tweets favoritados pelo usuário no momento da emissão do tweet
  # removido pois não parece seguir  linha temporal dos tweets
  # extracao_dados_user('favourites_count')

  #Número de usuários que são seguidos pelo usuário no momento da emissão do tweet
  # removido pois não parece seguir  linha temporal dos tweets
  # extracao_dados_user('friends_count')

  def extracao_entities():
         
    coluna_hashtags = []
    coluna_user_mentions = []

    for i in range(0,dataset_temp.shape[0]):
      list_hashtags = []
      list_user_mention = []
      if dataset_temp.loc[i]['entities'].get("hashtags") != None:
        for j in range(0,len(dataset_temp.loc[i]['entities'].get("hashtags"))):
          list_hashtags.append(dataset_temp.loc[i]['entities'].get("hashtags")[j].get('text'))
      coluna_hashtags.append(list_hashtags)

      if dataset_temp.loc[i]['entities'].get("user_mentions") != None:
        for j in range(0,len(dataset_temp.loc[i]['entities'].get("user_mentions"))):
         list_user_mention.append(dataset_temp.loc[i]['entities'].get("user_mentions")[j].get('name'))
      coluna_user_mentions.append(list_user_mention)

    dataset_temp['hashtags'] = coluna_hashtags
    dataset_temp['user_mentions'] = coluna_user_mentions

  extracao_entities()
  colunas_para_remocao = ['id_str', 'truncated', 'display_text_range', 'entities', 'lang',
                                   'source', 'in_reply_to_status_id','in_reply_to_status_id_str',
                                   'in_reply_to_user_id','in_reply_to_user_id_str', 'in_reply_to_screen_name',
                                   'user','geo','coordinates', 'place', 'contributors', 'is_quote_status',
                                   'possibly_sensitive', 'quoted_status_id','quoted_status_id_str',
                                  'quoted_status_permalink', 'extended_entities', 'quoted_status']
  dataset_temp = dataset_temp.drop(colunas_para_remocao, axis = 1)
  
  dataset_temp = dataset_temp.sort_values('created_at', ascending = True).reset_index()

  
  
  return dataset_temp

 


In [0]:
import pickle

lula_data = cleaning_data(dataset_lula)

bolso_data = cleaning_data(dataset_bolso)


with open("dados_lula.pickle", "wb") as file:
   pickle.dump(lula_data, file)

with open("dados_bolso.pickle", "wb") as file:
   pickle.dump(bolso_data, file)



In [316]:
bolso_data.tail()

,index,created_at,id,full_text,retweet_count,favorite_count,favorited,retweeted,character_count,followers_count,favourites_count,friends_count,hashtags,user_mentions
6789,2920,2020-01-08 15:27:39,1214931678730358784,- A lei 1.079 de 1950 trata dos crimes de resp...,3481,20617,False,False,178,5759095,2393,394,[],[]
6790,1444,2020-01-08 15:27:40,1214931682207371266,"- Não foi a Lei Orçamentária que criou o ""Fund...",1383,8330,False,False,179,5759090,2393,394,[],[]
6791,4397,2020-01-08 15:27:40,1214931683755134977,"- Do exposto, o veto aos R$ 2 bilhões, daria m...",1468,8877,False,False,199,5759095,2393,394,[],[]
6792,5971,2020-01-08 20:24:41,1215006428810924032,- Donald Trump e o Irã:\n\n- Nossa Constituiçã...,7581,48230,False,False,95,5759099,2393,394,[],[]
6793,674,2020-01-08 21:57:59,1215029910990774272,LEI ROMEO MION/Espectro Autista\n\n- Sancionad...,6269,40861,False,False,271,5759088,2393,394,[],[]


In [0]:
from datetime import datetime

#@title Opções para geração dos gráficos
#@markdown Forms support many types of fields.

Lula = True #@param {type:"boolean"}
Bolsonaro = True #@param {type:"boolean"}
data_inicial = '2010-04-01'  #@param {type: "date"}
data_final = '2020-08-01'  #@param {type: "date"}
if data_final < data_inicial:
  data_final = data_inicial
#@markdown ---


import matplotlib.pyplot as plt

bolso_dataset_sliced = bolso_data.loc[bolso_data['created_at'] >= datetime.strptime(data_inicial, '%Y-%m-%d').datetime() and
                                      bolso_data['created_at'] <= datetime.strptime(data_final, '%Y-%m-%d').date()]
lula_dataset_sliced = lula_data.loc[(lula_data['created_at'] >= datetime.strptime(data_inicial, '%Y-%m-%d').date()) and
                                    (lula_data['created_at'] <= datetime.strptime(data_final, '%Y-%m-%d').date())]


# if data_final > data_inicial:
#   bolso_data = bolso_data.loc[bolso_data['created_at'] >= data_inicial and bolso_data['created_at'] <= data_final]

plt.hist(lula_data['created_at'],10)
plt.hist(bolso_data['created_at'],10)
#plt.plot(lula_data['created_at'], lula_data['followers_count'])

plt.show()

In [454]:
import re
import string
from nltk.corpus import stopwords as sw
from string import punctuation
import nltk
from nltk.tokenize import regexp_tokenize
from nltk.tokenize import TweetTokenizer
def limpar_combinar_texto(dataset):
    
    nltk.download("stopwords")  
    
    stopwords =sw.words('portuguese')#+ list(punctuation))
   
    '''Takes a list of text and combines them into one large chunk of text.'''
    combined_text = ' '.join(dataset.full_text)
    combined_text = combined_text.lower()
   
    #combined_text = re.sub('\[.,!*?\]', '', combined_text)
    combined_text = re.sub('[%s]' % re.escape(string.punctuation), '', combined_text)
    combined_text = re.sub('\w*\d\w*', '', combined_text)
    combined_text = re.sub('http+', '', combined_text)
    combined_text = re.sub('[‘’“”…]', '', combined_text)
    combined_text = re.sub('\n', '', combined_text)
    tknzr = TweetTokenizer()
    alltokens = tknzr.tokenize(combined_text)
    combined_text = ' '.join([w for w in alltokens if not w in stopwords])
 
    return combined_text

texto_lula = limpar_combinar_texto(lula_data)
texto_bolso = limpar_combinar_texto(bolso_data)




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [455]:
texto_bolso

'boa noite todos depchicolanecar reportercrime desculpem boa noite depchicoalencar reportercrime militares assumiram governo março d apelos imprensa igreja católica mulheres passeatas brasil povo ñ suportava greves desordem social corrupção generalizada governo fez enérgico q pegaram armas acusavam militares violentos buscando votos indenizações milionárias fidel castro financiava luta armada idiota pode acreditar q diabo ditador queria implantar democracia brasil entreviste agora pessoas c d anos dirão q saudades governos militares médici ovacionado maracanã esquerda treinada cuba ñ podia receber outro tratamento pcc d hj pode ser combatido s energia brasil cuba mil psol chico votou contra moção apoio liberdade presos politicos cubanos financiados fidel posam d vítimas busca d votos indenizações censura mídia abole símblos religiosos etc brasil exportanto novelas gays ganhando bbbs psol defendendo ñ extradição criminoso battisti vivemos império violência corrupção nenhum militar enriq

In [468]:

# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
data_cv = cv.fit_transform([texto_bolso])
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())

data_dtm.transpose().sort_values(0, ascending= False)[:100]

,0
brasil,1256
bolsonaro,805
todos,588
governo,387
sobre,366
...,...
janeiro,86
acima,86
alguns,86
pode,86


In [467]:
cv = CountVectorizer()
data_cv = cv.fit_transform([texto_lula])
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())

data_dtm.transpose().sort_values(0, ascending= False)[:10]

,0
lula,5282
país,1705
brasil,1480
povo,1325
lulapelobrasil,1153
ser,987
presidente,925
gente,924
pra,842
haddadfernando,731


In [409]:
texto_lula.toarray()[0][2]

1

In [410]:
# Import the necessary modules




TypeError: ignored

In [384]:
alltokens

['lula',
 'acaba',
 'de',
 'chegar',
 'ao',
 'smabc',
 'de',
 'onde',
 'começará',
 'caminhada',
 'junto',
 'com',
 'dilmabr',
 'lulaedilmanoabc',
 'acompanhe',
 'notícias',
 'da',
 'caminhada',
 'também',
 'pelo',
 'ptbrasil',
 'e',
 'mudamais',
 'lulaedilmanoabc',
 'lula',
 'visita',
 'o',
 'smabc',
 'e',
 'cumprimenta',
 'sindicalistas',
 'e',
 'funcionários',
 'ao',
 'lado',
 'de',
 'dilmabr',
 'lulaedilmanoabc',
 'dilmabr',
 'e',
 'lula',
 'saem',
 'agora',
 'pelas',
 'ruas',
 'de',
 'são',
 'bernardo',
 'do',
 'campo',
 'e',
 'são',
 'saudados',
 'pela',
 'população',
 'lulaedilmanoabc',
 'os',
 'trabalhadores',
 'estão',
 'com',
 'dilmabr',
 'diz',
 'o',
 'prefeito',
 'de',
 'são',
 'bernardo',
 'do',
 'campo',
 'luiz',
 'marinho',
 'marinhopt',
 'à',
 'população',
 'lulaedilmanoabc',
 'mais',
 'de',
 'mil',
 'pessoas',
 'acompanham',
 'dilmabr',
 'e',
 'lula',
 'pelas',
 'ruas',
 'de',
 'são',
 'bernardo',
 'lulaedilmanoabc',
 'vocês',
 'geraram',
 'milhões',
 'de',
 'empregos'